<header>
    <h1>Section #1/6</h1>
</header>

<header>
    <h1>Import Libraries </h1>
</header>

In [ ]:
import pandas as pd
import psycopg2
import sqlalchemy
from sqlalchemy import create_engine
import numpy as np
import matplotlib.pyplot as plt
from dbConnstring import *

In [2]:
import dask
import distributed
import dask.array as da
import os
import time

In [3]:
pid=os.getpid()
print(pid)

3348


In [4]:
# Function to check path and edit:

def update_path():
    import os
    import sys
    #
    # Check path
    if not ( os.path.abspath('./bin') in  "__".join( sys.path ) ):
        print("Path needs to be updated: Adding to Path")
        sys.path.insert(0, os.path.abspath('./bin'))
    else:
        print('Path already contains...:', os.path.abspath('./bin'))
        #print('\n', "__".join( sys.path ))

In [5]:
update_path()

Path needs to be updated: Adding to Path


<header>
    <h1>Define Database Connection Details </h1>
</header>

In [6]:
# TimescaleDB username, password, and database name
TimescaleDB_USERNAME = '' ## YOUR TimescaleDB_USERNAME = 'abc'
TimescaleDB_PASSWORD = '' ## YOUR TimescaleDB_PASSWORD = 'xyz'
# Create the connection
postgres_str =  conn_str(TimescaleDB_USERNAME, TimescaleDB_PASSWORD)
cnx = create_engine(postgres_str)

<header>
    <h1>Section #2/6 </h1>
</header>

<header>
    <h1>Create a Delayed Function without a DASK Cluster </h1>
</header>

In [7]:
%%time 

import dask.dataframe as dd

df = dd.read_sql_table('hrv_500', 
                      postgres_str, 
                      npartitions=8, 
                      index_col='sensortime')  
df # by default it is a delayed

CPU times: user 185 ms, sys: 14.1 ms, total: 199 ms
Wall time: 1.08 s


,hr,rmssd,pid
npartitions=8,,,
2019-12-31 16:00:13,float64,float64,object
2020-02-03 11:14:28,...,...,...
...,...,...,...
2020-08-24 06:39:58,...,...,...
2020-09-27 01:54:16,...,...,...


<header>
    <h1>Verify it is just a Delayed Function without a DASK Cluster </h1>
</header>

In [8]:
%%time
df['hr'].describe() #.compute()

CPU times: user 25 ms, sys: 907 µs, total: 25.9 ms
Wall time: 24.9 ms


Dask Series Structure:
npartitions=1
    float64
        ...
Name: hr, dtype: float64
Dask Name: describe-numeric, 100 tasks

<header>
    <h1>Exectue the Delayed Function without a DASK Cluster </h1>
</header>

In [9]:
%%time
df.compute()

CPU times: user 35.3 s, sys: 25.2 s, total: 1min
Wall time: 37.4 s


,hr,rmssd,pid
sensortime,,,
2019-12-31 18:40:32,84.0,21.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 16:51:32,82.0,19.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 16:56:32,85.0,22.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 18:40:32,84.0,21.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 18:45:32,84.0,30.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
...,...,...,...
2020-08-27 17:11:17,69.0,18.0,W2aIXmOEC-T_GTPmqR2tjJ0NAxEkiNpXu-OQiirELQUuoN...
2020-08-27 17:16:17,69.0,18.0,W2aIXmOEC-T_GTPmqR2tjJ0NAxEkiNpXu-OQiirELQUuoN...
2020-08-27 17:21:17,70.0,18.0,W2aIXmOEC-T_GTPmqR2tjJ0NAxEkiNpXu-OQiirELQUuoN...


In [10]:
%%time
df['hr'].describe().compute()

CPU times: user 35.4 s, sys: 19.3 s, total: 54.6 s
Wall time: 36.4 s


count    6.920455e+06
mean     6.223385e+01
std      1.091681e+01
min      3.200000e+01
25%      5.500000e+01
50%      6.200000e+01
75%      7.000000e+01
max      1.500000e+02
Name: hr, dtype: float64

In [11]:
df.head()

,hr,rmssd,pid
sensortime,,,
2019-12-31 18:40:32,84.0,21.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 16:51:32,82.0,19.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 16:56:32,85.0,22.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 18:40:32,84.0,21.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 18:45:32,84.0,30.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...


<header>
    <h1>Section #3/6 </h1>
</header>

<header>
    <h1>Create a DASK Cluster</h1>
</header>

In [12]:
from dask import dataframe as dask_cluster_dd

In [13]:
N_WORKERS = 8

In [14]:
from dask.distributed import Client, LocalCluster

if __name__ == '__main__':
    # Create a Dask Cluster
    cluster = LocalCluster( n_workers=N_WORKERS, threads_per_worker=1, processes=True)
    client  = Client(cluster)

In [15]:
client

Client Scheduler: tcp://127.0.0.1:40939 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 8 Cores: 8 Memory: 135.06 GB


<header>
    <h1>Create a 'delayed' function with DASK Cluster</h1>
</header>

In [16]:
df = dask_cluster_dd.read_sql_table('hrv_500', 
                              postgres_str, 
                              npartitions=8, 
                              index_col='sensortime')  
df

,hr,rmssd,pid
npartitions=8,,,
2019-12-31 16:00:13,float64,float64,object
2020-02-03 11:14:28,...,...,...
...,...,...,...
2020-08-24 06:39:58,...,...,...
2020-09-27 01:54:16,...,...,...


<header>
    <h1>Exectue the 'delayed' function with DASK cluster</h1>
</header>

In [17]:
%%time
df['hr'].describe().compute() # time information is wrt 8cores

CPU times: user 1.07 s, sys: 157 ms, total: 1.23 s
Wall time: 13.7 s


count    6.920455e+06
mean     6.223385e+01
std      1.091681e+01
min      3.200000e+01
25%      5.500000e+01
50%      6.200000e+01
75%      7.000000e+01
max      1.500000e+02
Name: hr, dtype: float64

In [18]:
%%time
df.compute() # time information is wrt 8cores, 32GB

CPU times: user 3.49 s, sys: 2.62 s, total: 6.1 s
Wall time: 18 s


,hr,rmssd,pid
sensortime,,,
2019-12-31 18:40:32,84.0,21.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 16:51:32,82.0,19.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 16:56:32,85.0,22.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 18:40:32,84.0,21.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
2019-12-31 18:45:32,84.0,30.0,amX0V8J0Y0FVr9nOXLnyvftOMqmPdfys4-BFxceekvnsx3...
...,...,...,...
2020-08-24 20:21:47,47.0,61.0,W2aIXmOEC-T_GTPmqR2tjJ0NAxEkiNpXu-OQiirELQUuoN...
2020-08-24 20:26:47,48.0,59.0,W2aIXmOEC-T_GTPmqR2tjJ0NAxEkiNpXu-OQiirELQUuoN...
2020-08-24 20:31:47,48.0,40.0,W2aIXmOEC-T_GTPmqR2tjJ0NAxEkiNpXu-OQiirELQUuoN...


<header>
    <h1>Section #4/6 </h1>
</header>

<header>
    <h1>[4] Get Sktech HRV table</h1>
</header>

In [19]:
dask_data_hrv_skv = pd.read_sql_query('SELECT * FROM hrv_hr_skv_500 LIMIT 10;', cnx)
dask_data_hrv_skv

,per20,per40,per60,per80,variance,skewness,kurtosis,time,pid,granularity
0,32.0,32.0,32.0,32.0,0.0,None,None,2020-03-21 06:00:00,cf3lPK4nWnSZERpyQGAZ2z9UViyrYBw8WsDolSFHvD8BYG...,hourly
1,32.0,32.0,32.0,32.0,0.0,None,None,2020-04-20 11:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
2,32.0,32.0,32.0,32.0,0.0,None,None,2020-04-24 06:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
3,32.0,32.0,32.0,32.0,0.0,None,None,2020-04-24 12:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
4,32.0,32.0,32.0,32.0,0.0,None,None,2020-04-30 12:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
5,32.0,32.0,32.0,32.0,0.0,None,None,2020-04-30 13:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
6,32.0,32.0,32.0,32.0,0.0,None,None,2020-05-21 05:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
7,32.0,32.0,32.0,32.0,0.0,None,None,2020-05-21 06:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
8,32.0,32.0,32.0,32.0,0.0,None,None,2020-05-21 07:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly
9,32.0,32.0,32.0,32.0,0.0,None,None,2020-05-22 06:00:00,lZ3zOHrJHwMUTptP6aIlqRQMrjAssccgqxmkQeavagNCxk...,hourly


In [20]:
%%time 

import dask.dataframe as dd

df = dd.read_sql_table('hrv_hr_skv_500', 
                      postgres_str, 
                      npartitions=8, 
                      index_col='time')  
df # by default it is a delayed

CPU times: user 69.5 ms, sys: 9.62 ms, total: 79.2 ms
Wall time: 608 ms


,per20,per40,per60,per80,variance,skewness,kurtosis,pid,granularity
npartitions=8,,,,,,,,,
2019-12-31 00:00:00,float64,float64,float64,float64,float64,object,object,object,object
2020-02-02 21:07:30,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
2020-08-24 03:52:30,...,...,...,...,...,...,...,...,...
2020-09-27 01:00:00,...,...,...,...,...,...,...,...,...


In [21]:
%%time
df.compute() # 8cores TRUE

CPU times: user 927 ms, sys: 358 ms, total: 1.29 s
Wall time: 4.39 s


,per20,per40,per60,per80,variance,skewness,kurtosis,pid,granularity
time,,,,,,,,,
2020-01-02 12:00:00,34.0,34.0,34.0,34.0,0.000000,NaN,NaN,mLE7W4dw1WqhAaAPrp5b6aSNttQ2gj4bz9aeJw_BL0Fuif...,hourly
2020-01-02 13:00:00,34.0,34.0,34.0,34.0,0.000000,NaN,NaN,mLE7W4dw1WqhAaAPrp5b6aSNttQ2gj4bz9aeJw_BL0Fuif...,hourly
2020-02-02 06:00:00,34.0,34.0,34.0,34.0,NaN,NaN,NaN,65AB-VLz6t4ZX2V4P5oHkh2qorL7cR9HhWVYUN0LEa0YZM...,hourly
2020-01-02 14:00:00,35.0,35.0,36.0,36.0,0.500000,0,-0.0909091,BjTPWEt8ipbAxQXSh3PwvoxthXRVaBomWx5GpKqEmWHzfM...,hourly
2020-01-06 13:00:00,35.0,35.0,36.0,37.0,1.809524,0.271677,2.17943,BjTPWEt8ipbAxQXSh3PwvoxthXRVaBomWx5GpKqEmWHzfM...,hourly
...,...,...,...,...,...,...,...,...,...
2020-09-10 16:00:00,105.0,106.0,106.0,108.0,2.333333,0.381802,1.73964,Z_8F59UoVNgWATEuJQrJG2u_QQCrYjPWg4E9bUoTveyZKS...,hourly
2020-09-05 21:00:00,105.0,106.0,107.0,107.0,1.600000,-0.649519,1.96721,uWMADUpjb7SaUo_9gy2FYsv6qN1hdSRdC27YPM_ecy4-KP...,hourly
2020-09-05 00:00:00,105.0,106.0,107.0,111.0,12.666667,0.561588,1.75411,FGdRd4cQPVsXpv8N63ENL3KI0zriLdswiM9IFb5OaT2m8u...,hourly


<header>
    <h1>Section #5/6: Help </h1>
</header>

<header>
    <h1>Ask for Nautilus Account / Support on RocketChat</h1>
</header>

https://rocket.nautilus.optiputer.net/home

<header>
    <h1>Section #6/6: Extras</h1>
</header>

<header>
    <h1>Extras 1: Pull Data using URL </h1>
</header>

In [22]:
url_filename='https://raw.githubusercontent.com/fivethirtyeight/data/master/us-weather-history/KCLT.csv'

In [23]:
##https://nextcloud.nautilus.optiputer.net/s/WTFwMENM8sWm2dy/download

#url_filename= 'https://nextcloud.nautilus.optiputer.net/s/WTFwMENM8sWm2dy/download'

In [24]:
''' Dask reading from URL CSV File '''
bigdf  = dask_cluster_dd.read_csv(url_filename).compute()

In [25]:
bigdf.shape

(365, 13)

In [26]:
bigdf.groupby('record_max_temp_year').record_precipitation.mean()

record_max_temp_year
1879    1.230000
1881    2.460000
1887    1.637500
1888    1.950000
1889    1.462500
          ...   
2011    2.454286
2012    3.215000
2013    2.190000
2014    1.820000
2015    1.894000
Name: record_precipitation, Length: 102, dtype: float64

<header>
    <h1>Shutdown the DASK Cluster</h1>
</header>

In [27]:
client.shutdown()

<header>
    <h1>The End </h1>
</header>